In [ ]:
import sys
import os

sys.path.append(os.path.abspath('../../src'))

In [ ]:
from data_cleaner import DataCleaner
datapath = "../../../data/"
datafile = datapath+'binance_data_BTCFDUSD_20240528183200.parquet'
cleaner = DataCleaner(datafile, datapath, print_info=True)
cleaned_data = cleaner.clean_data()

In [ ]:
from create_target import TargetCalculator

In [ ]:
datafile = datapath+'data_cleaned.parquet'
calculator = TargetCalculator(datafile,datapath, print_info=True)
calculator.calculate_pct_change_target(time=120)
calculator.save_data('data_cleaned_target.parquet')

In [ ]:
from model_gen import ModelGenerator
datafile = datapath+'data_cleaned_target.parquet'
generator = ModelGenerator(datafile, datapath, print_info=True)
generator.create_train_test_split(test_size=0.3, shuffle=False)

# Train Logistic Regression
log_reg_model = generator.train_logistic_regression()
generator.evaluate_model(log_reg_model, generator.X_test, generator.y_test)
generator.save_model(log_reg_model, datapath+'log_reg_model.pkl')

In [ ]:
# Apply PCA and train Random Forest
pca = generator.apply_pca(n_components=20)
rf_model = generator.train_random_forest(n_estimators=100, random_state=42, pca = True)
generator.evaluate_model(rf_model, generator.X_test_pca, generator.y_test)

In [ ]:
generator.save_model(rf_model, datapath+'log_reg_model.pkl')

In [ ]:
y_pred_prob = log_reg_model.predict_proba(generator.X_test)

In [ ]:
from visu import DataVisualization
vis = DataVisualization(log_reg_model, generator.X_test, generator.y_test, y_pred_prob)

# Maintenant, vous pouvez utiliser les méthodes de visualisation
vis.plot_roc_curve()

In [ ]:
vis.plot_probability_density()

In [ ]:
vis.plot_calibration_curve()

In [ ]:
y_pred = log_reg_model.predict(generator.X_test)
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(generator.y_test, y_pred)

# Noms des classes (dans ce cas, classification binaire 0 et 1)
class_names = [0, 1]  # ou ['Negative', 'Positive'] par exemple
vis.plot_confusion_matrix(conf_matrix, class_names)